In [29]:
import os

import numpy as np
import torch
from src import train, predict
from src.utils import plot_fit_figures
import traceback
from src.models import LSTM
from sklearn.model_selection import ParameterGrid
import itertools
from src.utils import eval_func
from tqdm import tqdm

In [30]:
# 超参数的可能值
best_params = {
    'hidden_size': 56,
    'num_layers': 2,
    'batch_size': 64,
    'seq_len': 64,
}

hidden_size_options = [64]
num_layers_options = [2]
batch_size_options = [64]
seq_len_options = [64]
# 所有超参数组合
param_grid = list(itertools.product(hidden_size_options, num_layers_options, batch_size_options, seq_len_options))

In [ ]:
def run(best_config: dict, fund_code: str):
    model = LSTM(input_size=7,
                 output_size=7,
                 hidden_size=best_config['hidden_size'],
                 num_layers=best_config['num_layers'],
                 )
    config = {
        'model': model,
        'fund_code': fund_code,
        'data_set_length': 1000,
        'batch_size': best_config['batch_size'],
        'num_epochs': 60,
        'seq_len': best_config['seq_len'],
    }
    train(**config)
    config.pop('num_epochs')
    config.pop('data_set_length')
    predictions, groundtruths = predict(**config)
    plot_fit_figures(fund_code=fund_code, predictions=predictions, groundtruths=groundtruths)
    mse, mae, rmse, smape = eval_func(predictions, groundtruths)
    print(f'mse: {mse}, mae: {mae}, rmse: {rmse}, smape: {smape}')
    return predictions, groundtruths


In [31]:
run(best_params, fund_code='000079')

-------数据集加载中-------
Dataset type: train, data length: 726
Dataset type: valid, data length: 231
--------加载完成---------
----开始在cuda上训练------


训练中: 217003: 100%|██████████| 60/60 [00:01<00:00, 49.98it/s]

--------训练完成---------
-----loss曲线绘制完成-----
-------模型保存完成-------


In [32]:
config.pop('num_epochs')
config.pop('data_set_length')
predictions, groundtruths = predict(**config)

D:\Workplace\PythonProjects\IntelliFund\src\predict.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'checkpoints/{fund_code}.pt')


Dataset type: test, data length: 1621
---------开始预测---------


100%|██████████| 11/11 [00:00<00:00, 466.81it/s]

---------预测完成---------
---------评估完成---------


In [33]:

plot_fit_figures(fund_code=fund_code, predictions=predictions, groundtruths=groundtruths)


In [34]:
def grid_search(param_grid, fund_code):
    with open('grid_search_results.csv', 'w', encoding="utf-8") as f:
        f.write('hidden_size,num_layers,batch_size,seq_len,mse,mae,rmse,smape\n')
        for param in tqdm(param_grid):
            hidden_size, num_layers, batch_size, seq_len = param
            model = LSTM(input_size=7,
                         output_size=7,
                         hidden_size=hidden_size,
                         num_layers=num_layers,
                         )
            config = {
                'model': model,
                'fund_code': fund_code,
                'data_set_length': 1000,
                'batch_size': batch_size,
                'num_epochs': 60,
                'seq_len': seq_len,
            }
            try:
                train(**config)
                config.pop('num_epochs')
                config.pop('data_set_length')
                predictions, groundtruths = predict(**config)
                plot_fit_figures(fund_code=fund_code, predictions=predictions, groundtruths=groundtruths)
                mse, mae, rmse, smape = eval_func(predictions, groundtruths)
                f.write(f'{hidden_size},{num_layers},{batch_size},{seq_len},{mse},{mae},{rmse},{smape}\n')

            except Exception as e:
                print(param)
                print(traceback.format_exc())
                continue


In [35]:
grid_search(param_grid, fund_code)

  0%|          | 0/6 [00:00<?, ?it/s]

-------数据集加载中-------
Dataset type: train, data length: 707
Dataset type: valid, data length: 177
--------加载完成---------
----开始在cuda上训练------



训练中: 217003: 100%|██████████| 60/60 [00:02<00:00, 26.72it/s]


--------训练完成---------
-----loss曲线绘制完成-----
-------模型保存完成-------


D:\Workplace\PythonProjects\IntelliFund\src\predict.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'checkpoints/{fund_code}.pt')


...

D:\Workplace\PythonProjects\IntelliFund\src\predict.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'checkpoints/{fund_code}.pt')


Dataset type: test, data length: 1757
---------开始预测---------



100%|██████████| 23/23 [00:00<00:00, 430.27it/s]

---------预测完成---------
---------评估完成---------



100%|██████████| 6/6 [00:19<00:00,  3.20s/it]
